In [1]:
# Step 1
#Linear_SVC model

In [2]:
# Step 2
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

data = load_iris()
X = data['data']
y = data['target']
df = pd.DataFrame(np.c_[X,y], columns=data['feature_names']+['target'])

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [3]:
df = df.drop(columns='sepal length (cm)')
df = df.drop(columns='sepal width (cm)')

# Categorize the target at good quality wine (1) or bad quality wine (0)
def categorizeQualityData(dataFrame, columnName):
    for i in range(dataFrame.shape[0]):
        if dataFrame[columnName][i] == 2:
            dataFrame[columnName][i] = 1
        else:
            dataFrame[columnName][i] = 0  
    return 

categorizeQualityData(df, 'target')

df.tail()

,petal length (cm),petal width (cm),target
145,5.2,2.3,1.0
146,5.0,1.9,1.0
147,5.2,2.0,1.0
148,5.4,2.3,1.0
149,5.1,1.8,1.0


In [4]:
df = df.sample(frac=1) 
df.head()

,petal length (cm),petal width (cm),target
13,1.1,0.1,0.0
128,5.6,2.1,1.0
25,1.6,0.2,0.0
92,4.0,1.2,0.0
68,4.5,1.5,0.0


In [5]:
# Step 3
def standardizeData(dataSet):
    for key, values in dataSet.iteritems():
        mean = dataSet[key].mean()
        std = dataSet[key].std()
        if key != 'target':
            dataSet[key] = (dataSet[key] - mean) / std
    return dataSet

standardizeData = standardizeData(df)

# Splitting the data into Train Set and Test Set
def partition(X,y,t):
    x_train = np.array(X[int((len(X)+1)*t):])
    x_test = np.array(X[:int((len(X)+1)*t)])
    y_train = np.array(y[int((len(y)+1)*t):])
    y_test = np.array(y[:int((len(y)+1)*t)]) 
    return x_train, x_test, y_train, y_test

#separate the target vector and features
def separateTargetVector(dataSet):
    y = dataSet['target'] #target vector
    X = dataSet.drop(columns='target') #features matrix
    return X,y

#a function to split data into partition
#according to the number of folds
def partitionSplit(df, folds):
    s_partition = []
    dfCopy = df
    #set the size for each partiton
    eachPartition = int(df.shape[0] / folds)
    for i in range(folds):
        partition = []
        #going through each partition
        for j in range(eachPartition):
            if(len(partition) < eachPartition):
                value = dfCopy.index[0]
                partition.append(value.tolist())
                dfCopy = dfCopy.drop(value)
        s_partition.append((np.asarray(partition)).tolist())
  
    return s_partition

In [ ]:
# Step 4

#function for the s-fold validation
def sFold(folds, data, labels, model, error_function, dictionary): 
    avgAccuracy = [] 
    value = []
    splitX = partitionSplit(data, folds)
    minError = 100.0
    
    #loop to choose a partition as validation set
    for i, partition in enumerate(splitX):
        trainSet = list(splitX)
        del(trainSet[i])       #dropping the validation set
        trainSet = sum(trainSet, [])

        testSet = []           #getting the validation set 
        for row in partition:
            testSet.append(row)  #put data in validation set into a new list 
            
        #splitting the data into train and test set
        xTrainValues = []
        yTrainValues = []        
        for trainIndex in trainSet:
            xTrainValues.append(data.loc[trainIndex])
            yTrainValues.append(labels.loc[trainIndex])

        #splitting the validation set data into train and test set
        xTestValues = []
        yTestValues = []
        for testIndex in testSet:
            xTestValues.append(data.loc[testIndex])
            yTestValues.append(labels.loc[testIndex])
    
        #preform prediction using Softmax Regression
        if model == 'Linear_SVC':
            modelClassifier = Linear_SVC(C=1, max_iter=100, tol=None, learning_rate=‘constant’,learning_rate_init=0.001, t_0=1, t_1=1000, early_stopping=False, validation_fraction=0.1)
            modelClassifier.fit(np.array(xTrainValues), np.array(yTrainValues))
            yPredict= modelClassifier.predict(np.array(xTestValues))
            
            #calculating the error using mse score
            if error_function == "accuracy":
                accuracy_value = accuracy(yPredict, np.array(yTestValues))
                avgAccuracy.append(accuracy_value)
                
    avgFinalAccuracy = sum(avgAccuracy)/folds
    if(avgFinalAccuracy < minError):
        minError = avgFinalAccuracy

    return minError
    
    
#function to pass the hyperparameter into s-fold validation
def dictionary(lambd, tol, learning_rate, regularizer):
    result = []
    model = []
    modelArgs = dict()
    #form the dictionary
    for eachLambd in lambd:
        for eachTol in tol:
            for eachLearningRate in learning_rate:
                for eachRegularizer in regularizer:
                    modelArgs = {'lambd' : eachLambd, 'tol' : eachTol, 'learning_rate' : eachLearningRate,'regularizer': eachRegularizer}
                    #use of s-folds
                    accuracy = sFold(5, X, y, 'Linear_SVC', 'accuracy', modelArgs)
                    result.append(accuracy)
                    model.append(modelArgs)

    acc = np.array(result)
    best_args = model[acc.argmax()]
    return best_args
    
#get the X data(features without target) and y data(target column)
X,y = separateTargetVector(standardizeData)

C = []
learning_rate = []
learning_rate_init(when ‘constant’ learning_rate is used)
max_iter = []
tol = []

best_args = dictionary()
print(best_args)

In [ ]:
# Step 5
X,y = separateTargetVector(standardizeData)
xTrain, xTest, yTrain, yTest = partition(X, y, 0.2)
modelClassifier = Linear_SVC(C=1, max_iter=100, tol=None, learning_rate=‘constant’,learning_rate_init=0.001, t_0=1, t_1=1000, early_stopping=False, validation_fraction=0.1)
modelClassifier.fit(xTrain, yTrain)
yPredict = modelClassifier.predict(xTest)
yPredict = np.array(yPredict)

print('Accuracy:', accuracy(yPredict, yTest))
print('Confusion Matrix:', confusionMatrix(yTest, yPredict))
print("\n")

In [ ]:
import math

def partitionSplit(trainSetSize, folds):
    s_partition = []
    dfCopy = df
    #set the size for each partiton
    eachPartition = int(len(trainSetSize) / folds)
    for i in range(folds):
        partition = []
        #going through each partition
        for j in range(eachPartition):
            if(len(partition) < eachPartition):
                value = dfCopy.index[0]
                partition.append(value.tolist())
                dfCopy = dfCopy.drop(value)
        s_partition.append((np.asarray(partition)).tolist())
  
    return s_partition

def trainDataSize(df, train_size):
    s_partition = []
    trainSize = train_size
    dataSize = int(df.shape[0])

    while trainSize < dataSize:
        partition = []
        for i in range(trainSize):
            if(len(partition) < trainSize):
                    value = df.index[i]
                    partition.append(value.tolist())
        s_partition.append((np.asarray(partition)).tolist())
        trainSize+=train_size
  
    return s_partition

def learning_curve(model, X, Y, cv, train_size=10, learning_rate=0.01, epochs=1000, tol=None, regularizer=None, lambd=0.0):
    finalTrainErrorSet = []
    finalValidationErrorSet = []
    trainSetSize = trainDataSize(X, train_size)
    
    for i in range(len(trainSetSize)):
        x = trainSetSize[i]
        splitX = partitionSplit(x, cv)
        train_scores = [] 
        val_scores = []
        #loop to choose a partition as validation set
        for i, partition in enumerate(splitX):
            trainSet = list(splitX)

            del(trainSet[i])       #dropping the validation set
            trainSet = sum(trainSet, [])

            validationSet = []           #getting the validation set 
            for row in partition:
                validationSet.append(row)  #put data in validation set into a new list

            #splitting the data into train and test set
            xTrainValues = []
            yTrainValues = []     
            for trainIndex in trainSet:
                xTrainValues.append(X.loc[trainIndex])
                yTrainValues.append(Y.loc[trainIndex])

            #splitting the validation set data into train and test set
            xValidationValues = []
            yValidationValues = []
            for testIndex in validationSet:
                xValidationValues.append(X.loc[testIndex])
                yValidationValues.append(Y.loc[testIndex])
                
            #perform prediction using Linear Regression
            if model == 'Linear_Regression':
                modelRegression = Linear_Regression()
                modelRegression.fit(np.array(xTrainValues), np.array(yTrainValues), learning_rate, epochs, tol, regularizer,lambd)
                
                yTestPredict = modelRegression.predict(np.array(xValidationValues))
                validation_error = math.sqrt(mse(yValidationValues, yTestPredict))
                val_scores.append(validation_error)
                
                yTrainPredict = modelRegression.predict(np.array(xTrainValues))
                train_error = math.sqrt(mse(yTrainValues, yTrainPredict))
                train_scores.append(train_error)
                
        avgFinalTrainError = sum(train_scores)/cv
        finalTrainErrorSet.append(avgFinalTrainError)
        
        avgFinalValidationError = sum(val_scores)/cv
        finalValidationErrorSet.append(avgFinalValidationError)
        
    return finalTrainErrorSet, finalValidationErrorSet
    

In [ ]:
# Step 6

import matplotlib.pyplot as plt
%matplotlib inline

X,y = separateTargetVector(standardizeData)
trainSize = 20
train_scores, val_scores= learning_curve("Linear_Regression", X, y, 5, train_size=trainSize, learning_rate=0.01, 
                                          epochs=1000, tol=None, regularizer="l2", lambd=0.0001)

x = []
i = trainSize
while i < len(X):
    x.append(i)
    i = i + trainSize

#Plotting
fig = plt.figure(figsize=(10, 6))
plt.plot(x, val_scores, color='red', linewidth=5, linestyle='--')
plt.plot(x, train_scores, color='blue', linewidth=5, linestyle='--') 
plt.title('Best Model Plot')
plt.xlabel('Traning Set Data')
plt.ylabel('RSME')
plt.show()


In [ ]:
# Step 7
#plot decision boundary